In [1]:
# set env via .env file
import sys
import os
import pandas as pd
import datetime
import time
import munch
import shared
import config
import model
import provider_yfinance as provider


In [2]:
print('init> initialize environment')
config.print_env()

init> initialize environment
TF version: 2.1.0
Keras version: 2.2.4-tf
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is there a GPU available: False
Is the Tensor on GPU #0: False
Device name: /job:localhost/replica:0/task:0/device:CPU:0
Eager Execution enabled: True


In [3]:
cfg = config.get_config('^GDAXI', True)
# recalc specific date
# config.overwrite_end_dt(cfg, '2020-02-03')
config.save_config(cfg)

config> current directory:/mnt/d/notebooks/sandbox
config> data period: from '2018-03-26' to '2020-02-24'
config> download period: from '2018-01-08' to '2020-02-24'
shared> parsing stocks from web '^GDAXI' ...
config> benchmarks: '['000001.SS', 'AUDUSD=X', 'BTCUSD=X', 'CNY=X', 'ETHUSD=X', 'EURCAD=X', 'EURCHF=X', 'EURGBP=X', 'EURHUF=X', 'EURJPY=X', 'EURJPY=X', 'EURSEK=X', 'EURUSD=X', 'GBPJPY=X', 'GBPUSD=X', 'GC=F', 'HKD=X', 'IDR=X', 'IMOEX.ME', 'INR=X', 'JPY=X', 'KW=F', 'LC=F', 'LH=F', 'MXN=X', 'MYR=X', 'NZDUSD=X', 'PA=F', 'PHP=X', 'PL=F', 'RUB=X', 'S=F', 'SGD=X', 'SI=F', 'SM=F', 'THB=X', 'ZAR=X', '^AORD', '^AXJO', '^BFX', '^BSESN', '^BUK100P', '^BVSP', '^DJI', '^FCHI', '^FTSE', '^GDAXI', '^GSPC', '^GSPTSE', '^HSI', '^IPSA', '^IXIC', '^JKSE', '^JN0U.JO', '^KLSE', '^KS11', '^MERV', '^MXX', '^N100', '^N225', '^NYA', '^NZ50', '^RUT', '^STI', '^STOXX50E', '^TA125.TA', '^TWII', '^VIX', '^XAX']'
config> stocks: '['1COV.DE', 'ADS.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BEI.DE', 'BMW.DE', 'CON.DE'

In [ ]:
%%time
# total: 1h
# compact: 5m

start_ts = time.time()
print(f'download> start downloading data {cfg.prepare.data_end_dt} ...')

cfg_stocks, data_stocks = provider.load_stocks(cfg, compact=True)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg, compact=True)

print(f'download> download finished, duration: {time.time() - start_ts:.2f} s')

download> start downloading data 2020-02-24 ...
shared> failed to load pickle file: '/mnt/d/notebooks/sandbox/cache/20200224/stocks_1d.pkl'
shared> failed to load pickle file: '/mnt/d/notebooks/sandbox/cache/20200224/stocks_1d_meta.pkl'
tickers-stocks> downloading histories ...
[*********************100%***********************]  30 of 30 completed
WARN: tickers-stocks> #missing: 34675
ticker-1COV.DE> loading ticker data ...
ticker-ADS.DE> loading ticker data ...
ticker-ALV.DE> loading ticker data ...
ticker-BAS.DE> loading ticker data ...
ticker-BAYN.DE> loading ticker data ...
ticker-BEI.DE> loading ticker data ...
WARN: ticker-BEI.DE> #missing: 10
ticker-BMW.DE> loading ticker data ...
ticker-CON.DE> loading ticker data ...
yf-cashflowStatementHistory> internal error: Internal Server Error
yf-cashflowStatementHistoryQuarterly> internal error: Internal Server Error
yf-balanceSheetHistory> internal error: Internal Server Error
yf-balanceSheetHistoryQuarterly> internal error: Internal S

In [ ]:
%%time 
# 5 s

start_ts = time.time()
print(f'prepare> preparing stock and benchmark data ...')

cfg_stocks, data_stocks = provider.load_stocks(cfg)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg)

prep_stocks = provider.prepare_stocks(cfg, data_stocks)
prep_benchmarks = provider.prepare_benchmarks(cfg, data_benchmarks)

enc_stocks = provider.encode_stocks(cfg, prep_stocks)
enc_benchmarks = provider.encode_benchmarks(cfg, prep_benchmarks, prep_stocks)

In [ ]:
%%time 
# total: 20min - 50min
# compact: 20min

print(f'prepare> preparing submodel data ...')
for submodel_settings in cfg.train.settings:
    print(f"sm-{submodel_settings.id}> preparing submodel data ...")
    model_data = provider.prepare_submodel_data(cfg, submodel_settings, enc_stocks, enc_benchmarks)
    # update num_features setting (informational)
    submodel_settings.num_features = len(model_data.X[0][0][0][0])
config.save_config(cfg)

In [ ]:
duration = time.time() - start_ts
print(f'prepare> preparation finished, duration: {duration:.2f} s')

In [ ]:
%%time

start_ts = time.time()
print(f'train> training started ...')

print('train> list all submodule settings')
[(i, s.id) for i, s in enumerate(cfg.train.settings)]

model.train_full(cfg, start_settings_idx=0)

duration = time.time() - start_ts
print(f'train> training finished, duration: {duration:.2f} s')

In [ ]:

all_subdirs = [os.path.join('/mnt/d/notebooks/sandbox/model/', d) for d in os.listdir('/mnt/d/notebooks/sandbox/model/') if os.path.isdir(os.path.join('/mnt/d/notebooks/sandbox/model/', d)) and not d.startswith('.')]
latest_subdir = max(all_subdirs, key=os.path.getmtime)
latest_subdir

In [ ]:
%%time

start_ts = time.time()
print(f'predict> prediction started ...')

predictions = model.predict(cfg)

duration = time.time() - start_ts
print(f'predict> prediction finished, duration: {duration:.2f} s')

In [ ]:
os.listdir('/mnt/d/notebooks/sandbox/model/')